In [9]:
import numpy as np
import math
import random
from matplotlib import pyplot as plt

In [8]:
# For Monte Carlo Approach

for i in range(10):
    print(random.uniform(1,10))

9.549721931483209
1.4701909239845508
9.579804944240557
1.0658251623608335
3.600117098677355
9.435883971864476
3.6851372024371707
5.670227818627141
5.072191996125119
7.251281930766034


In [13]:
# For Simulation Assuming Distributions

for i in range(10):
    print(random.gauss(0,1))

1.090032024403626
0.8371630823865888
2.157955695099237
-2.7063966493803546
-1.8206652091076283
1.5212949624732819
-0.5270236939415323
-1.1154462205692968
0.3971036225086773
-0.404906679268235


In [8]:
# Duty Cycle

# Variables Definition
Vout =  np.array([3.2, 3.3, 3.4])# Vpre_static [3.2, 3.3, 3.4]V
LS_Ron = 0.01 # Rds_on1 [6.3, 8.3, 11] mOhms
Iout = 3.45 # Iload_3V3M [2.64, 3.03, 3.45]
Vin = 13.37 # Vpre_in [13.3, 13.35, 13.37]V -> INPUT!!!!
HS_Ron = LS_Ron #Rds_on1 

# Formula
D = (Vout + (LS_Ron * Iout))/(Vin - (HS_Ron * Iout) + (LS_Ron * Iout))
D

# Expected Result for: VBat_range = 13.5V -> D3v3 = [0.242, 0.249, 0.257]

0.2568810770381451

In [11]:
Vout =  np.array([3.2, 3.3, 3.4])# Vpre_static [3.2, 3.3, 3.4]V

array([3.2, 3.3, 3.4])